In [1]:
import json
import sys
from pathlib import Path
import os

import numpy as np

from utils import *

from sklearn.model_selection import train_test_split
from transformers import AutoConfig, AutoTokenizer, AutoModel

from tqdm import tqdm

# import keras
# import tensorflow as tf
# from keras.callbacks import CSVLogger, ModelCheckpoint
# from keras.layers import Input
# from keras.models import Model

import torch
from torch.utils.data import DataLoader
from transformers import AdamW
from torch.nn import functional as F


In [2]:
HW_path = Path(r'F:\Daneshgah\Arshad\2_4002\NLP\HW5')

data_folder = HW_path / 'data'
results_folder = HW_path / 'results'

cache_dir = Path(r'F:\Daneshgah\Arshad\2_4002\NLP\HW5\cache') #HW_path / 'cache'

# config = AutoConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased", cache_dir=cache_dir)

## Data

# x_train = np.load(data_folder / 'persica.csv_train_x.npy', allow_pickle=True).tolist()
# y_train = np.load(data_folder / 'persica.csv_train_y.npy').tolist()
x_test = np.load(data_folder / 'persica.csv_test_x.npy', allow_pickle=True).tolist()
y_test = np.load(data_folder / 'persica.csv_test_y.npy').tolist()

In [3]:
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased", cache_dir=cache_dir)
max_length = 512 #max([len(tokenizer.tokenize(x)) for x in x_train]) + 2

test_encodings = tokenizer(x_test, truncation=True, padding=True, max_length=max_length)


In [4]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

test_dataset = IMDbDataset(test_encodings, y_test)

In [5]:
model_number = (results_folder / 'last_model_number.txt').read_text().strip()
model_path_dir = results_folder / model_number
model_path = model_path_dir / 'model.pt'
hyperparameters_path = model_path_dir / 'hyperparameters.json'
model_path_dir

WindowsPath('F:/Daneshgah/Arshad/2_4002/NLP/HW5/results/13')

In [6]:
with open(hyperparameters_path, 'r') as file:
     hyperparameters = json.loads(file.read())
     
num_labels = hyperparameters['num_labels']
device = 'cuda'
batch_size = hyperparameters['batch_size']
model_name = hyperparameters['model_name']
model_name

'HooshvareLab/bert-base-parsbert-uncased'

In [7]:
base_model = AutoModel.from_pretrained(model_name, cache_dir=cache_dir, num_labels=num_labels)
model = CustomTorchModel(base_model, num_labels)
model.to(device)
model.load_state_dict(torch.load(model_path))
model.eval()


Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CustomTorchModel(
  (base_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(100000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [9]:
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


In [26]:
loss = 0
counter = 0

for batch in tqdm(test_loader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    token_type_ids = batch['token_type_ids'].to(device)
    labels = F.one_hot(batch['labels'], num_classes=num_labels).float().to(device)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels, token_type_ids=token_type_ids)
    if int(torch.argmax(outputs)) != 9:
        print(int(torch.argmax(outputs)), int(torch.argmax(labels)))
    
    counter += 1
    


  0%|          | 0/2088 [00:00<?, ?it/s]f:\Daneshgah\Arshad\2_4002\NLP\HW5\code\utils.py:337: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = self.softmax(outputs)
 10%|▉         | 207/2088 [00:06<00:56, 33.46it/s]


KeyboardInterrupt: 

In [27]:
outputs

tensor([[-2.4285, -2.4704, -2.3875, -2.4231, -2.2637, -2.5013, -2.2762, -2.5193,
         -2.3723, -2.2544, -2.5330]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward0>)

In [16]:
loss

tensor(2.4231, device='cuda:0', grad_fn=<DivBackward1>)